In [1]:
import sys

sys.path.insert(0, '../03_Remake_with_TDD/algorithms')
from ht_time import *

sys.path.insert(0, '../03_Remake_with_TDD/postgresql')
from ht_player_postgresql import *
from ht_match_postgresql import *

In [2]:
ht_time = HattrickTime()
folder = ht_time.findLastSunOrWed()
# folder = ht_time.findLastSunOrWed('2018/04/04')
# folder = ht_time.findLastSunOrWed('2018/06/13')
# folder = "2018/07/01"
print("folder =", folder)

folder = 2018/10/10


In [ ]:
folder = "2018/10/03"
print("folder =", folder)

In [3]:
import re
from bs4 import BeautifulSoup

In [4]:
import sys

sys.path.insert(0, '../97_Parse/')
from parseMatch import *

sys.path.insert(0, '../98_Crawler/01_DualMonitor/')
from DualMonitor import getIsDualMonitor

sys.path.insert(0, '../98_Crawler/02_WebBrowser/')
from FirefoxTab import clickTabOfFireFox
from crawMatch import *

sys.path.insert(0, '../98_Crawler/03_OnScreen/')
from Empty import clickEmpty
from MoveScroll import moveScroll
from MoveFindMoveClick import moveByImage, moveByImageWithModify

sys.path.insert(0, '../98_Crawler/04_Login/')
# from HattrickLogInOut import clickFacebookLogin, clickFacebookLogout
from HattrickLogInOut import clickHattrickLogin2, clickHattrickLogout

In [34]:
# isDualMonitor
isDualMonitor = getIsDualMonitor()

# Hattrick LastKeeper Tab
clickTabOfFireFox(5, isDualMonitor)

# Hattrick Login by Facebook
# clickEmpty(410, isDualMonitor)
clickEmpty(305, isDualMonitor)
moveScroll(20)
# clickFacebookLogin(484, 395, isDualMonitor)
clickHattrickLogin2(500, 305, isDualMonitor)

# # Hattrick Logout by Facebook
# clickEmpty(336, isDualMonitor)
# moveScroll(20)
# clickFacebookLogout(1258, 336, isDualMonitor)

use 2 monitor = 3840 x 1080 isDualMonitor= True


In [ ]:
# isDualMonitor
isDualMonitor = getIsDualMonitor()

# Hattrick LastKeeper Tab
clickTabOfFireFox(5, isDualMonitor)

# Hattrick Login by Facebook
# clickEmpty(410, isDualMonitor)
clickEmpty(305, isDualMonitor)
moveScroll(20)
# clickFacebookLogin(484, 394, isDualMonitor)
clickHattrickLogin2(500, 305, isDualMonitor)

# Hattrick MyClub
moveScroll(20)
moveByImage(530, 340, 77, 42, isDualMonitor, 'screen-MyClub.png')

# Hattrick Match
clickEmpty(700, isDualMonitor)
moveScroll(20)
moveByImage(384, 696, 80, 108, isDualMonitor, 'screen-Match.png')

# # Hattrick LastMatch
clickEmpty(831, isDualMonitor)
moveScroll(20)
# # moveByImageWithModify(1040, 831, 15, -15, 58, 48, isDualMonitor, 'screen-LastMatch.png')
moveAndClick(1082, 837, isDualMonitor)
time.sleep(5)

# FirefoxFile
moveAndClick(90, 15, isDualMonitor)

# SavePageAs
moveAndClick(91, 130, isDualMonitor)

# Type match
time.sleep(1)
typeFileName()

# rwoo folder
moveAndClick(570, 460, isDualMonitor)

# 02_workspace folder
moveAndClicks(720, 440, isDualMonitor, 2)

# 04_Hattrick folder
moveAndClicks(720, 500, isDualMonitor, 2)

# HattrickStudy
moveAndClicks(720, 415, isDualMonitor, 2)

# 00_LastKeeper
moveAndClicks(720, 415, isDualMonitor, 2)

# 2018
moveAndClicks(720, 440, isDualMonitor, 2)

# Last Month
clickLastMonthOrDay()

# Last Day
clickLastMonthOrDay()

# Save
moveByImage(1280, 847, 91, 41, isDualMonitor, 'screen-Save.png')

# Hattrick Logout by Facebook
clickEmpty(336, isDualMonitor)
moveScroll(20)
# clickFacebookLogout(1258, 337, isDualMonitor)
clickHattrickLogout(1270, 370, isDualMonitor)

# Jupyter Tab
clickTabOfFireFox(7, isDualMonitor)

In [5]:
from ht_file import *
HattrickFile = HattrickFile()

In [6]:
HattrickFile.check_and_delete_folder(folder + '/match_files')

2018/10/10/match_files		 : is not exitsed


In [7]:
from ht_match import *

In [8]:
HattrickMatch = HattrickMatch()
ht_player_table_name = 'player_tmp'

In [9]:
match_dict_list = HattrickMatch.findMatchList(folder + '/match.html', 'mydatabase', ht_player_table_name)

Home


In [10]:
match_str_list = HattrickMatch.matchDictListToMatchStrList(match_dict_list)

print("\ncat match_str_List\n")
for str_line in match_str_list:
    print(str_line)


cat match_str_List

po,num,rt,sMin,eMin
KP,2,8.0,0,90
WB,61,4.5,0,90
WB,78,4.5,0,70
WB,6,4.0,70,90
CD,70,5.5,0,90
CD,4,4.5,0,75
CD,72,4.0,75,90
W,15,5.0,0,90
W,8,4.5,0,55
W,65,4.0,55,90
IM,12,5.0,0,90
IM,25,5.0,0,90
FW,16,5.5,0,90
FW,31,4.5,0,90


In [11]:
HattrickFile.check_and_delete_file(folder + '/match.txt')

2018/10/10/match.txt	 : is not existed


In [12]:
for line in match_str_list:
    with open (folder + '/match.txt', 'a') as f:
        f.write(line + "\n")

In [13]:
import psycopg2

In [14]:
conn = psycopg2.connect("dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'")
conn

<connection object at 0x7f185c086178; dsn: 'user=myuser password=xxx dbname=mydatabase host=localhost port=65432', closed: 0>

In [ ]:
# def dropMatchTable():
#     conn = None
#     try:
#         conn = psycopg2.connect("dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'")
#         cur = conn.cursor()
#         sql = """DROP TABLE IF EXISTS match ; """
#         cur.execute(sql)
#         conn.commit()
#     except(Exception, psycopg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()

In [ ]:
#dropMatchTable()

In [ ]:
# def createMatchTable():
#     conn = None
#     try:
#         conn = psycopg2.connect("dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'")
#         cur = conn.cursor()
#         sql = """
# CREATE TABLE match (
#     Date DATE,          --01
#     Po   VARCHAR(5),    --02 Position
#     Num  INT,           --03 Number
#     RT   NUMERIC(2, 1), --04 Rating
#     sMin INT,           --05 Start Min
#     eMin INT            --06 End Min
# ) ; """
#         cur.execute(sql)
#         conn.commit()
#     except(Exception, psycopg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()

In [ ]:
# createMatchTable()

In [15]:
hattrickMatchPostgresql = HattrickMatchPostgreSQL()
hattrickPlayerPostgresql = HattrickPlayerPostgreSQL()

In [16]:
hattrickMatchPostgresql.select_count_of_match(conn)

932

In [17]:
import csv
with open(folder + '/match.txt', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        if(re.match( '^[0-9]{1,2}', row[1] ) ) :
            hattrickMatchPostgresql.insert_to_match(conn, folder.replace('/', '-'), row)

In [18]:
hattrickMatchPostgresql.select_count_of_match(conn)

946

In [19]:
hattrickMatchPostgresql.update_rt_of_match(conn, ht_player_table_name)

In [20]:
tupleList = hattrickMatchPostgresql.getMatchByDate(folder, conn)
hattrickMatchPostgresql.printMatchByDate(tupleList)

KP	2	7.5	0	90
WB	78	4.5	0	70
WB	6	4.0	70	90
WB	61	4.5	0	90
CD	4	4.5	0	75
CD	72	4.0	75	90
CD	70	5.0	0	90
W	8	4.5	0	55
W	65	4.0	55	90
W	15	5.0	0	90
IM	25	4.5	0	90
IM	12	5.0	0	90
FW	31	4.5	0	90
FW	16	5.5	0	90


In [21]:
HattrickFile.check_and_delete_file(folder + '/match.txt')

2018/10/10/match.txt	 : will be removed
2018/10/10/match.txt	 : is not existed


In [22]:
with open ( folder + '/match.txt', 'a' ) as f:
    f.write("po,num,rt,sMin,eMin\n")
    f.write( '\n'.join('%s,%s,%s,%s,%s' % myTuple for myTuple in tupleList) + '\n' )

In [23]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'KP', [2,76])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [2,76])

date num min/rt |    02   |    76   |
----------------|---------|---------|
2018-09-12      |  90/6.0 |         |
2018-09-16      |         |  90/4.5 |
2018-09-19      |  90/6.5 |         |
2018-09-23      |         |  90/4.5 |
2018-09-26      |  90/7.0 |         |
2018-09-30      |         |  90/5.0 |
2018-10-03      |  90/7.5 |         |
2018-10-07      |         |  90/5.0 |
2018-10-10      |  90/7.5 |         |


In [26]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'WB', [6,61,78,80,81])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [6,61,78,80,81])
#  6 -> backUp
# 14 -> out
tupleList = hattrickPlayerPostgresql.select(conn, ht_player_table_name, folder, [6,61,78,80,81], 'wb')
print('Num', 'b_p', 'b_p_v', 'wbd_p', 'wb_p', 'wbtm_p', 'wbo_p', sep='\t')
hattrickPlayerPostgresql.print(tupleList, sep='\t')

date num min/rt |    06   |    61   |    78   |    80   |    81   |
----------------|---------|---------|---------|---------|---------|
2018-09-12      |         |  15/5.0 |  75/4.5 |         |         |
2018-09-16      |  90/4.0 |         |         |         |         |
2018-09-19      |         |         |  65/4.5 |         |         |
2018-09-23      |         |  90/4.0 |         |         |         |
2018-09-26      |  62/4.5 |         |  90/4.5 |         |         |
2018-09-30      |         |         |         |  46/5.0 |  44/4.5 |
2018-10-03      |  20/4.0 |  90/4.5 |  70/4.5 |         |         |
2018-10-07      |  20/4.0 |         |         |  90/5.0 |  70/4.5 |
2018-10-10      |  20/4.0 |  90/4.5 |  70/4.5 |         |         |
Num	b_p	b_p_v	wbd_p	wb_p	wbtm_p	wbo_p
6	WBd	6.75	6.75	6.52	6.38	6.27
61	WBd	8.55	8.55	8.33	8.22	8.09
78	WBd	6.78	6.78	6.34	6.29	5.87
80	WBd	8.9	8.90	8.68	8.58	8.45
81	WBd	8.19	8.19	7.90	7.95	7.61


In [27]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'CD', [4,38,40,52,70,72])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [4,38,40,52,70,72])
#  4 cd_tw
# 38 cd
# 40 cd
# 52 cd
# 70 cd
# 72 cd backUp
tupleList = hattrickPlayerPostgresql.select(conn, ht_player_table_name, folder, [6,61,78,80,81], 'cd')
print('Num', 'b_p', 'b_p_v', 'wbd_p', 'wb_p', 'wbtm_p', 'wbo_p', sep='\t')
hattrickPlayerPostgresql.print(tupleList, sep='\t')

date num min/rt |    04   |    38   |    40   |    52   |    70   |    72   |
----------------|---------|---------|---------|---------|---------|---------|
2018-09-12      |         |         |  90/5.0 |         |  50/5.5 |  40/4.0 |
2018-09-16      |         |  90/4.0 |         |         |  38/4.5 |         |
2018-09-19      |  46/4.0 |         |  90/4.5 |         |         |  44/4.0 |
2018-09-23      |         |  90/4.0 |  90/4.5 |         |         |         |
2018-09-26      |  90/4.0 |         |         |         |  76/5.0 |         |
2018-09-30      |         |  90/4.0 |  90/5.5 |         |         |         |
2018-10-03      |  90/4.5 |         |         |         |         |  90/4.0 |
2018-10-07      |         |  44/3.5 |  90/5.5 |  46/5.0 |         |         |
2018-10-10      |  75/4.5 |         |         |         |  90/5.0 |  15/4.0 |
Num	b_p	b_p_v	wbd_p	wb_p	wbtm_p	wbo_p
6	WBd	6.75	6.21	6.56	5.62
61	WBd	8.55	8.08	8.39	7.53
78	WBd	6.78	6.23	6.62	5.29
80	WBd	8.9	8.44	8.75	7.9

In [28]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'W', [8,11,13,15,65])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [8,11,13,15,65])
#  8 w_m
# 11 w_m
# 65 w

date num min/rt |    08   |    11   |    13   |    15   |    65   |
----------------|---------|---------|---------|---------|---------|
2018-09-12      |         |         |  90/5.0 |  90/4.5 |         |
2018-09-16      |  65/5.0 |         |  90/4.5 |         |  25/4.5 |
2018-09-19      |         |         |  90/4.5 |  90/4.5 |         |
2018-09-23      |  60/4.5 |         |  83/4.0 |         |  30/4.0 |
2018-09-26      |  44/4.5 |  90/4.0 |         |         |  46/4.0 |
2018-09-30      |  50/4.5 |  90/4.0 |         |         |  40/4.0 |
2018-10-03      |  46/4.5 |         |         |  90/5.0 |  44/4.0 |
2018-10-07      |  44/4.0 |  90/4.0 |  46/3.5 |         |         |
2018-10-10      |  55/4.5 |         |         |  90/5.0 |  35/4.0 |


In [29]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'IM', [10,12,25,34,79])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [10,12,25,34,79])
# 34 -> backUp
# 79 -> im_d

date num min/rt |    10   |    12   |    25   |    34   |    79   |
----------------|---------|---------|---------|---------|---------|
2018-09-12      |         |  90/5.0 |  55/5.0 |         |         |
2018-09-16      |  90/5.0 |         |         |  90/4.0 |         |
2018-09-19      |         |  90/5.0 |  90/4.5 |         |         |
2018-09-23      |  90/4.5 |  10/5.0 |         |  80/4.0 |         |
2018-09-26      |         |  90/5.0 |  60/5.0 |         |  30/5.5 |
2018-09-30      |  90/4.0 |         |         |         |  90/6.0 |
2018-10-03      |         |  90/5.0 |  90/4.5 |         |         |
2018-10-07      |  90/3.5 |         |         |         |  90/6.0 |
2018-10-10      |         |  90/5.0 |  90/4.5 |         |         |


In [31]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'FW', [16,31,48,55])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [16,31,48,55])
# 16 fw_tw
# 31
# 48 fw_tw
# 55
tupleList = hattrickPlayerPostgresql.select(conn, ht_player_table_name, folder, [16,31,48,55], 'fw')
print('Num', 'b_p', 'b_p_v', 'fw_p', 'fwd_p', 'fwtw_p', 'tdf_p', sep='\t')
hattrickPlayerPostgresql.print(tupleList, sep='\t')

date num min/rt |    16   |    31   |    48   |    55   |
----------------|---------|---------|---------|---------|
2018-09-12      |         |  90/4.5 |         |  90/5.0 |
2018-09-16      |  90/4.5 |         |  90/4.0 |         |
2018-09-19      |  24/4.5 |  66/4.5 |         |  90/5.0 |
2018-09-23      |  46/5.0 |         |  44/3.5 |  90/4.5 |
2018-09-26      |  90/4.5 |         |         |  90/4.5 |
2018-09-30      |  70/5.5 |         |  20/3.0 |  90/4.5 |
2018-10-03      |  70/5.5 |         |  20/2.5 |  90/4.5 |
2018-10-07      |  90/5.5 |         |         |  90/4.0 |
2018-10-10      |  90/5.5 |  90/4.5 |         |         |
Num	b_p	b_p_v	fw_p	fwd_p	fwtw_p	tdf_p
16	FWtw	7.27	6.97	6.20	7.27	0.00
31	FW	5.98	5.98	5.44	5.98	0.00
48	FWtw	4.89	4.83	0.00	4.89	4.88
55	IMd	6.08	6.08	5.69	5.92	0.00


In [32]:
conn.close()
print(conn)

<connection object at 0x7f185c086178; dsn: 'user=myuser password=xxx dbname=mydatabase host=localhost port=65432', closed: 1>


In [33]:
conn = None
print(conn)

None
